In [1]:
from src.models.lstm_based.base_model import epsilon_model
from src.models.structures import *
from src.models.layers.image_layers import *
from src.models.lstm_based.classification_heads import *
from src.models.lstm_based.helper import *

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow_addons as tfa

import sklearn as sk
import numpy as np

In [2]:
BUFFER = 2048
BATCH_SIZE = 128

In [3]:
K = 1000
EPSILON = [0.001, 0.005, 0.01, 0.05, 0.1]
EPOCHS = 1
SEED = 8008

In [4]:
PARTITION_DIR = "D:\SUMMER_2022\PROJECT\PredictionsFromAggregations\data\interim\lstm"
DATASET = 'MNIST'
DIR = "D:\SUMMER_2022\PROJECT\PredictionsFromAggregations\models\v1"

In [5]:
name, data = retrieve_dataset(DATASET, None)
x_train, x_test, y_train, y_test = data
dataset = Dataset(name, x_train, x_test, y_train, y_test)

In [6]:
flat_dim = (None, np.product(dataset.x_train.shape, dtype=np.int32) // len(dataset.x_train))

In [7]:
mean, dataset = aggregate(dataset, K, PARTITION_DIR, SEED)

In [8]:
n_classes = len(np.unique(dataset.y_train))

flatten_batch = lambda x : x.reshape(x.shape[0], np.product(x.shape[1:], dtype=np.int32))

x_train = flatten_batch(dataset.x_train)
y_train = tfk.utils.to_categorical(dataset.y_train, n_classes)
x_test = flatten_batch(x_test)
y_test = tfk.utils.to_categorical(dataset.y_test, n_classes)

In [9]:
"""I present the current worst function in the codebase"""
tf_convert = lambda x, y, type : (tf.data.Dataset.from_tensor_slices((tf.cast(x, type), tf.cast(y, type)))).shuffle(BUFFER).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

train_set = tf_convert(x_train, y_train, tf.uint8)
test_set = tf_convert(x_test, y_test, tf.uint8)

In [10]:
merger = tfk.layers.Bidirectional(tfk.layers.LSTM(mean.astype(int), activation='relu', name='merging_layer'))

In [11]:
config = generator_config(flat_dim, 10, n_classes, None, None, merger)
model = epsilon_model(config)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
[10000, 15000], [1e-0, 1e-1, 1e-2])
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)

optim = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
loss_func = tfk.losses.CategoricalCrossentropy()
metrics = [None]

model.compile(optimizer=optim, loss=loss_func, metrics=metrics)

In [12]:
history = model.fit(dataset.x_train, dataset.y_train, epochs=EPOCHS, validation_split=0.2)

UnboundLocalError: in user code:

    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Parry\AppData\Local\Temp\__autograph_generated_file763whrkt.py", line 10, in tf__call
        intermediate = ag__.converted_call(ag__.ld(tf).concat, ([ag__.convert